In [18]:
import pandas as pd
import openai
import os
from tqdm import tqdm

from openai import OpenAI, OpenAIError


In [19]:
# openai.api_type = "azure"
# openai.api_base = "https://lit-azure-openai-2.openai.azure.com/"
# openai.api_version = "2023-09-01-preview"
# openai.api_key =  "sk-proj-srf0GOOUxV7oOvgszSjPT3BlbkFJcER9kB6xb0wUq6x9N0Iq"
os.environ["OPENAI_API_KEY"] = "sk-proj-srf0GOOUxV7oOvgszSjPT3BlbkFJcER9kB6xb0wUq6x9N0Iq"
# api_key =  "sk-proj-srf0GOOUxV7oOvgszSjPT3BlbkFJcER9kB6xb0wUq6x9N0Iq"
client = OpenAI()

In [20]:
completion = client.chat.completions.create(
  model="gpt-4-turbo-2024-04-09",
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Hello!"}
  ]
)

print(completion.choices[0].message.content)

Hello! How can I assist you today?


In [21]:
def get_completion(prompt):
    messages = [{"role": "user", "content": prompt}]
    try:
        response = client.chat.completions.create(
            model="gpt-4-turbo-2024-04-09",  # Using the specific model as indicated
            messages=messages,
            temperature=1,
            top_p=1
        )
        return response.choices[0].message.content  # Accessing the 'content' using dot notation
    except OpenAIError as e:
        print(f"OpenAI Error: {e} - There was an issue with the API call.")
        return None  # Handle OpenAI API specific errors
    except Exception as e:
        print(f"General Error: {e} - An unexpected error occurred.")
        return None  # Handle other unforeseen errors

# Example usage
prompt = "hello!"
output_sentence = get_completion(prompt)
print(output_sentence if output_sentence else "Prompt was skipped or an error occurred.")


# from openai import OpenAI
# client = OpenAI()

# completion = client.chat.completions.create(
#   model="gpt-4-turbo-2024-04-09",
#   messages=[
#     {"role": "system", "content": "You are a helpful assistant."},
#     {"role": "user", "content": "Hello!"}
#   ]
# )

# print(completion.choices[0].message)

Hello! How can I assist you today?


In [22]:
def load_examples(file_path):
    with open(file_path, 'r') as file:
        examples = file.read().strip()
    return examples

In [23]:
def generate_data(dataset_csv, examples_file, output_csv):
    # Read the dataset CSV
    df = pd.read_csv(dataset_csv)
    # df = df.head(5)

    examples = load_examples(examples_file)
    # Initialize lists to store explanations and predictions
    
    
    explanations = []
    predictions = []
    
    # Iterate over each row in the dataset
    for index, row in tqdm(df.iterrows(), total=len(df), desc="Generating predictions"):
        # Extract the text from the row
        text = row['text']
        
        # Generate the completion using the provided prompt and get_completion function
        prompt = f"{examples}\n\n" \
                 f"Please follow the INSTRUCTION and think step by step: given a SENTENCE, first output an explanation about whether you think the given SENTENCE is labeled as abuse, then output the label PREDICTION\n\nINSTRUCTION: Think step by step: given the following SENTENCE, first output an explanation about whether you think the given SENTENCE is labeled as abuse, then output the label PREDICTION\nSENTENCE: {text}\nEXPLANATION: \nPREDICTION: "
        completion = get_completion(prompt)
        
        # Extract the explanation and prediction from the completion
        explanation = completion.split("EXPLANATION: ")[-1].split("PREDICTION:")[0].strip()
        prediction = completion.split("PREDICTION:")[-1].strip()

        if prediction == "NONE":
            prediction = "NONE ABUSE"
        
        # Append the explanation and prediction to their respective lists
        explanations.append(explanation)
        predictions.append(prediction)
    
    # Add explanations and predictions as columns to the DataFrame
    df['explanations'] = explanations
    df['predictions'] = predictions
    display(df)
    # Save the DataFrame with explanations and predictions
    df.to_csv(output_csv, index=False)

In [24]:
dataset_csv = '../../dataset/train_dev/test_set.csv'
# output_csv = 'gpt4_cot_prediction_test.csv'
output_csv = 'gpt4_cot_prediction.csv'
examples_file = 'cot_FewShot_prompt.txt'

In [25]:
generate_data(dataset_csv, examples_file, output_csv)

Generating predictions: 100%|██████████| 360/360 [20:17<00:00,  3.38s/it]


,inst_id,text,labels,explanations,predictions
0,1,You take things that don't belong to you.,ABUSE,This sentence is labeled as abuse because it d...,ABUSE
1,2,I take it you don't want to be characterised i...,NONE ABUSE,This sentence is labeled as not abusive becaus...,NONE ABUSE
2,3,You are so imaginative at what could go wrong.,NONE ABUSE,This sentence is labeled as not abusive becaus...,NONE ABUSE
3,4,You need to act your age.,ABUSE,This sentence is labeled as abuse because it u...,ABUSE
4,5,I think you might be challenged vertically.,ABUSE,This sentence is labeled as abuse because it u...,ABUSE
...,...,...,...,...,...
355,356,"You really carry on talking, regardless of the...",NONE ABUSE,This sentence is labeled as abuse because it s...,ABUSE
356,357,You are the reason I have decided to spend mor...,NONE ABUSE,This sentence is labeled as not abusive becaus...,NONE ABUSE
357,358,You have always given me amazing vibes.,NONE ABUSE,This sentence is labeled as not abusive becaus...,NONE ABUSE
358,359,It's great that no one notices you when you sh...,NONE ABUSE,This sentence is labeled as abuse because it s...,ABUSE


In [26]:
from sklearn.metrics import classification_report

def evaluate(predictions_path):
    # Load the predictions dataframe
    predictions_df = pd.read_csv(predictions_path)
    
    # Extract true labels and predicted labels
    true_labels = predictions_df['labels']
    predicted_labels = predictions_df['predictions']
    
    # Generate classification report
    report = classification_report(true_labels, predicted_labels, output_dict=True)
    
    # Extract precision, recall, and F1 score for each class
    precision = report['weighted avg']['precision']
    recall = report['weighted avg']['recall']
    f1_score = report['weighted avg']['f1-score']
    
    # Print evaluation metrics
    print("Evaluation Metrics:")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1_score:.4f}")

evaluate(output_csv)

Evaluation Metrics:
Precision: 0.8366
Recall: 0.8361
F1 Score: 0.8358


/home/longju/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/longju/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/longju/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
